# Geodatenanalyse 2


## Termin 10 - Modul 2

## *Earth Engine*: Analyse und Export

In [1]:
import geemap
import geemap.colormaps as cm
import datetime as dt
import pandas as pd
import ee
# initialize the connection to the server
ee.Initialize()

## Übung 1

Das Höhenmodell [ETOPO1 (Global 1 Arc-Minute Elevation)](https://developers.google.com/earth-engine/datasets/catalog/NOAA_NGDC_ETOPO1) wird in dieser Übung verwendet. Zusätzlich verwenden wir noch die Länder des [US Department of State, Office of Geography](https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017):


- **Aufgabe 1**: Welche Kanäle enthält dieses Raster? Welche weiteren Eigenschaften hat dieses Raster?

- **Aufgabe 2**: Extrahiere das Polygon für die Schweiz aus der Länderliste.

- **Aufgabe 3**: Zeige dieses Höhenmodell und die Grenze der Schweiz in einer dynamischen Karte an.

- **Aufgabe 4**: Extrahiere ein Raster des Höhenmodells der Schweiz.

- **Aufgabe 5**: Berechne die Höhenstatistik (Minimum, Maximum, Durchschnitt) für die Schweiz.

- **Aufgabe 6**: Exportiere das Höhenmodell der Schweiz als GeoTIFF-Datei.


In [2]:
# AUFGABE 1

# das Höhenmodell
etopo = ee.Image('NOAA/NGDC/ETOPO1')
etopo.bandNames().getInfo()

['bedrock', 'ice_surface']

In [3]:
# AUFGABE 2
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
switzerland = countries.filter(ee.Filter.eq('country_na', 'Switzerland'))
switzerland.getInfo()

{'type': 'FeatureCollection',
 'columns': {'abbreviati': 'String',
  'country_co': 'String',
  'country_na': 'String',
  'system:index': 'String',
  'wld_rgn': 'String'},
 'id': 'USDOS/LSIB_SIMPLE/2017',
 'version': 1566850254549599,
 'properties': {'date_range': [1490832000000, 1490832000000],
  'period': 0,
  'thumb': 'https://mw1.google.com/ges/dd/images/LSIB_thumb.png',
  'description': '<p>The United States Office of the Geographer provides\nthe Large Scale International Boundary (LSIB) dataset. The detailed\nversion (2013) is derived from two other datasets: a LSIB line\nvector file and the World Vector Shorelines (WVS) from the National\nGeospatial-Intelligence Agency (NGA). The interior boundaries\nreflect U.S. government policies on boundaries, boundary disputes,\nand sovereignty. The exterior boundaries are derived from the\nWVS; however, the WVS coastline data is outdated and generally\nshifted from between several hundred meters to over a kilometer.\nEach feature is the pol

In [4]:
# AUFGABE 3

# Ergebnis anzeigen ...
bedrock = etopo.select('bedrock')
globe = geemap.Map()
vis = {'min': 0, 'max': 5000, 'palette': cm.palettes.dem}
globe.addLayer(bedrock, vis, 'etopo', opacity=0.7)

vis_params = {'color': "024ACA", 'width': 0.5}
globe.addLayer(switzerland, vis_params, name='Switzerland')

globe.centerObject(switzerland, 7)

globe

Map(center=[46.80116948995133, 8.226523584592684], controls=(WidgetControl(options=['position', 'transparent_b…

In [5]:
# AUFGABE 4

ch = bedrock.clip(switzerland)
ch.getInfo()

globe = geemap.Map()
vis = {'min': 0, 'max': 5000, 'palette': cm.palettes.dem}
globe.addLayer(ch, vis, 'etopo')
globe.centerObject(switzerland, 7)

globe

Map(center=[46.80116948995133, 8.226523584592684], controls=(WidgetControl(options=['position', 'transparent_b…

In [6]:
# AUFGABE 5

# a combination of reducers
red_comb = ee.Reducer.mean().combine(
  reducer2 = ee.Reducer.stdDev(),
  sharedInputs = True
).combine(
  reducer2 = ee.Reducer.min(),
  sharedInputs = True
).combine(
  reducer2 = ee.Reducer.max(),
  sharedInputs = True
)

# Min, max and standard deviation
stats = bedrock.reduceRegion(
    reducer = red_comb,
    geometry = switzerland,
    scale = 30,
    maxPixels = 1e9)

stats.getInfo()

{'bedrock_max': 4271,
 'bedrock_mean': 1304.0248754597278,
 'bedrock_min': 191,
 'bedrock_stdDev': 787.8603658993911}

In [7]:
# AUFGABE 6

geemap.ee_export_image(etopo, "data/CH_etopo.tif", region=switzerland.geometry(), file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to D:\KIT\3_lehre\2022_SS\Geodatenanalyse_2\kursmaterial\Termin-10_(Gabriel)\data


## Übung 2

Das European Centre for Medium-Range Weather Forecasts (ECMWF) stellt kontinuierlich Klimadaten zusammen. Ein Beispiel ist das [ERA5 'Daily aggregates' Archiv](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY)

- **Aufgabe 1**: Welche Kanäle enthält diese Rasterkollektion? Was sind die Einheiten der jeweiligen Rasterdaten?

- **Aufgabe 2**: Welcher Zeitbereich wird von diesem Datensatz abgedeckt?

- **Aufgabe 3**: Zeige die weltweite Temperaturverteilung für den 1. Januar 2020 in einer dynamischen Karte an.

- **Aufgabe 4**: Extrahiere die durchschnittliche Temperatur sowie den Regen für Karlsruhe als CSV-Datei für die verfügbare Zeit.

In [8]:
# AUFGABE 1

climate = ee.ImageCollection("ECMWF/ERA5/DAILY")
climate.first().bandNames().getInfo()

['mean_2m_air_temperature',
 'minimum_2m_air_temperature',
 'maximum_2m_air_temperature',
 'dewpoint_2m_temperature',
 'total_precipitation',
 'surface_pressure',
 'mean_sea_level_pressure',
 'u_component_of_wind_10m',
 'v_component_of_wind_10m']

In [9]:
climate.first().select('mean_2m_air_temperature').getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}],
 'version': 1577976434807309,
 'id': 'ECMWF/ERA5/DAILY/19790102',
 'properties': {'system:time_start': 284083200000,
  'month': 1,
  'year': 1979,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 284169600000,
  'system:asset_size': 36884548,
  'day': 2,
  'system:index': '19790102'}}

In [10]:
# AUFGABE 2

dates = climate.reduceColumns(ee.Reducer.toList(), ["system:time_start"]).get('list')
datelist = pd.to_datetime(dates.getInfo(), unit='ms')
datelist

DatetimeIndex(['1979-01-02', '1979-01-03', '1979-01-04', '1979-01-05',
               '1979-01-06', '1979-01-07', '1979-01-08', '1979-01-09',
               '1979-01-10', '1979-01-11',
               ...
               '2020-06-30', '2020-07-01', '2020-07-02', '2020-07-03',
               '2020-07-04', '2020-07-05', '2020-07-06', '2020-07-07',
               '2020-07-08', '2020-07-09'],
              dtype='datetime64[ns]', length=15165, freq=None)

In [11]:
# Aufgabe 3

temp_image = climate.filterDate('2020-01-01', '2020-01-02').select('mean_2m_air_temperature')
temp_image.first().getInfo()

globe = geemap.Map(zoom=1)
vis = {'min': 265, 'max': 300, 'palette': cm.palettes.jet}
globe.addLayer(temp_image, vis, 'precipitation', opacity=0.7)
globe

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [12]:
# AUFGABE 4

KA = ee.Geometry.Point(8.405, 49.014)
buffer = 100

# Get the data for the point in urban area.
KA_climate = climate.getRegion(KA, buffer).getInfo()
len(KA_climate)

15166

In [13]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [14]:
KA_climate_df = ee_array_to_df(KA_climate, ['mean_2m_air_temperature', 'total_precipitation'])

# konvertiere in Celsius
KA_climate_df['mean_2m_air_temperature'] -= 273.15

# konvertiere in mm
KA_climate_df['total_precipitation'] *= 1000

KA_climate_df

,time,datetime,mean_2m_air_temperature,total_precipitation
0,284083200000,1979-01-02,-10.205511,2.219412
1,284169600000,1979-01-03,-8.272009,0.123099
2,284256000000,1979-01-04,-11.066595,0.004849
3,284342400000,1979-01-05,-12.839575,0.000000
4,284428800000,1979-01-06,-12.705023,0.000000
...,...,...,...,...
15160,1593907200000,2020-07-05,22.939630,0.173679
15161,1593993600000,2020-07-06,17.528741,2.277711
15162,1594080000000,2020-07-07,17.146082,0.102550
15163,1594166400000,2020-07-08,18.139124,0.176672


In [15]:
# als CSV speichern
KA_climate_df.to_csv("data/KA_ERA5_example.csv")

## ENDE